In [1]:
%pip install numpy
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



In [ ]:
# In this section I am importing all the libraries I will need
import numpy as np
import matplotlib.pyplot as plt


In [ ]:



# In this section I am setting the domain of solution and the discretised grid



# In this section I am defining arrays I would need (if neeeded)



# In this section I am setting the boundary conditions/initial values




# In this section I am implementing the numerical method




# In this section I am showing the results



# In this section I am celebrating
print('CW done: I deserve a good mark')

